In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
import warnings
import requests
import re

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')

### WTI 원유선물

In [24]:
codename = 'WTI원유선물'

In [25]:
import MySQLdb
db = MySQLdb.connect(host = 'localhost', user = 'root', password = '1234')
cursor = db.cursor()

In [26]:
db.commit()

In [27]:
sql = """
    create database if not exists oil_price;
"""
cursor.execute(sql)

1

In [28]:
sql = """
    use oil_price;
"""
cursor.execute(sql)

0

In [29]:
sql = """
    drop table if exists daum;
"""
cursor.execute(sql)

0

In [30]:
sql = """
    create table if not exists daum (
        company char(20) not null,
        date char(10) not null,
        종가 float,
        전일비 float,
        등락률 float
    );
"""
cursor.execute(sql)

0

In [11]:
# 권한 오류로 개같이 멸망
# url = 'https://finance.daum.net/api/domestic/exchanges/COMMODITY-/CLc1/days?symbolCode=COMMODITY-%2FCLc1&page=1&perPage=30&fieldName=changeRate&order=desc&pagination=true'
# res_set = {
#     'symbolCode': 'COMMODITY-/CLc1',
#     'page' : '1',
#     'perPage': '30',
#     'fieldName': 'changeRate',
#     'order': 'desc',
#     'pagination': 'true'
# }
# requests.post(url, res_set).text


'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body bgcolor="white">\r\n<center><h1>403 Forbidden</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'

In [31]:
driver = webdriver.Chrome()

In [32]:

url = 'https://finance.daum.net/domestic/exchange/COMMODITY-%2FCLc1'
driver.get(url)
time.sleep(0.5)

In [33]:
for _ in range(1):
    for page in range(2,12):
        html = driver.page_source
        df = pd.read_html(html)[0]
        df['전일비'] = df['전일비'].str.replace('▼','-').str.replace('▲','').astype('float')
        df['등락률'] = df['등락률'].str.replace('%','').astype(float)
        for i in range(len(df)):
            sql = f"""
                insert into daum values ('{codename}','{df.iloc[i,0]}',{df.iloc[i,1]}, {df.iloc[i,2]}, {df.iloc[i,3]})"""
            cursor.execute(sql)
        path = f'div.paging > a:nth-child({page})'
        driver.find_element_by_css_selector(path).click()
        time.sleep(0.5)
db.commit()

In [34]:
for _ in range(26):
    for page in range(4,14):
        html = driver.page_source
        df = pd.read_html(html)[0]
        df['전일비'] = df['전일비'].str.replace('▼','-').str.replace('▲','').astype('float')
        df['등락률'] = df['등락률'].str.replace('%','').astype(float)
        for i in range(len(df)):
            sql = f"""
                insert into daum values ('{codename}','{df.iloc[i,0]}',{df.iloc[i,1]}, {df.iloc[i,2]}, {df.iloc[i,3]})"""
            cursor.execute(sql)
        path = f'div.paging > a:nth-child({page})'
        driver.find_element_by_css_selector(path).click()
        time.sleep(0.5)
db.commit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.paging > a:nth-child(13)"}
  (Session info: chrome=98.0.4758.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00AEFDC3+2555331]
	Ordinal0 [0x00A877F1+2127857]
	Ordinal0 [0x00982E08+1060360]
	Ordinal0 [0x009AE49E+1238174]
	Ordinal0 [0x009AE69B+1238683]
	Ordinal0 [0x009D9252+1413714]
	Ordinal0 [0x009C7B54+1342292]
	Ordinal0 [0x009D75FA+1406458]
	Ordinal0 [0x009C7976+1341814]
	Ordinal0 [0x009A36B6+1193654]
	Ordinal0 [0x009A4546+1197382]
	GetHandleVerifier [0x00C89622+1619522]
	GetHandleVerifier [0x00D3882C+2336844]
	GetHandleVerifier [0x00B823E1+541697]
	GetHandleVerifier [0x00B81443+537699]
	Ordinal0 [0x00A8D18E+2150798]
	Ordinal0 [0x00A91518+2168088]
	Ordinal0 [0x00A91660+2168416]
	Ordinal0 [0x00A9B330+2208560]
	BaseThreadInitThunk [0x7714FA29+25]
	RtlGetAppContainerNamedObjectPath [0x772F7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x772F7A6E+238]


In [35]:
db.commit()

In [36]:
pd.read_sql('select * from daum', db)

,company,date,종가,전일비,등락률
0,WTI원유선물,2022.02.09,89.66,0.30,0.34
1,WTI원유선물,2022.02.08,89.36,-1.96,-2.15
2,WTI원유선물,2022.02.07,91.32,-0.99,-1.07
3,WTI원유선물,2022.02.04,92.31,2.04,2.26
4,WTI원유선물,2022.02.03,90.27,2.01,2.28
...,...,...,...,...,...
8067,WTI원유선물,1990.01.08,21.62,-1.46,-6.33
8068,WTI원유선물,1990.01.05,23.08,-0.33,-1.41
8069,WTI원유선물,1990.01.04,23.41,-0.27,-1.14
8070,WTI원유선물,1990.01.03,23.68,0.79,3.45


### 두바이유

In [37]:
codename = '두바이유선물'

In [38]:
import MySQLdb
db = MySQLdb.connect(host = 'localhost', user = 'root', password = '1234')
cursor = db.cursor()

In [39]:
sql = """
    use oil_price;
"""
cursor.execute(sql)

0

In [40]:
url = 'https://finance.daum.net/domestic/exchange/COMMODITY-%2FDCc1'
driver.get(url)
time.sleep(0.5)

In [41]:
# 1~10
for _ in range(1):
    for page in range(2,12):
        html = driver.page_source
        df = pd.read_html(html)[0]
        df['전일비'] = df['전일비'].str.replace('▼','-').str.replace('▲','').astype('float')
        df['등락률'] = df['등락률'].str.replace('%','').astype(float)
        for i in range(len(df)):
            sql = f"""
                insert into daum values ('{codename}','{df.iloc[i,0]}',{df.iloc[i,1]}, {df.iloc[i,2]}, {df.iloc[i,3]})"""
            cursor.execute(sql)
        path = f'div.paging > a:nth-child({page})'
        driver.find_element_by_css_selector(path).click()
        time.sleep(0.5)
db.commit()

In [42]:
# 11~130
for _ in range(13):
    for page in range(4,14):
        html = driver.page_source
        df = pd.read_html(html)[0]
        df['전일비'] = df['전일비'].str.replace('▼','-').str.replace('▲','').astype('float')
        df['등락률'] = df['등락률'].str.replace('%','').astype(float)
        for i in range(len(df)):
            sql = f"""
                insert into daum values ('{codename}','{df.iloc[i,0]}',{df.iloc[i,1]}, {df.iloc[i,2]}, {df.iloc[i,3]})"""
            cursor.execute(sql)
        path = f'div.paging > a:nth-child({page})'
        driver.find_element_by_css_selector(path).click()
        time.sleep(0.5)
db.commit()

In [56]:
# 131~134
for _ in range(1):
    for page in range(10,16):
        html = driver.page_source
        df = pd.read_html(html)[0]
        df['전일비'] = df['전일비'].str.replace('▼','-').str.replace('▲','').astype('float')
        df['등락률'] = df['등락률'].str.replace('%','').astype(float)
        for i in range(len(df)):
            sql = f"""
                insert into daum values ('{codename}','{df.iloc[i,0]}',{df.iloc[i,1]}, {df.iloc[i,2]}, {df.iloc[i,3]})"""
            cursor.execute(sql)
        path = f'div.paging > a:nth-child({page})'
        driver.find_element_by_css_selector(path).click()
        time.sleep(0.5)
db.commit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.paging > a:nth-child(13)"}
  (Session info: chrome=98.0.4758.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00AEFDC3+2555331]
	Ordinal0 [0x00A877F1+2127857]
	Ordinal0 [0x00982E08+1060360]
	Ordinal0 [0x009AE49E+1238174]
	Ordinal0 [0x009AE69B+1238683]
	Ordinal0 [0x009D9252+1413714]
	Ordinal0 [0x009C7B54+1342292]
	Ordinal0 [0x009D75FA+1406458]
	Ordinal0 [0x009C7976+1341814]
	Ordinal0 [0x009A36B6+1193654]
	Ordinal0 [0x009A4546+1197382]
	GetHandleVerifier [0x00C89622+1619522]
	GetHandleVerifier [0x00D3882C+2336844]
	GetHandleVerifier [0x00B823E1+541697]
	GetHandleVerifier [0x00B81443+537699]
	Ordinal0 [0x00A8D18E+2150798]
	Ordinal0 [0x00A91518+2168088]
	Ordinal0 [0x00A91660+2168416]
	Ordinal0 [0x00A9B330+2208560]
	BaseThreadInitThunk [0x7714FA29+25]
	RtlGetAppContainerNamedObjectPath [0x772F7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x772F7A6E+238]


In [66]:
db.commit()

In [67]:
tmpdf =pd.read_sql('select * from daum', db)
tmpdf

,company,date,종가,전일비,등락률
0,WTI원유선물,2022.02.09,89.66,0.30,0.34
1,WTI원유선물,2022.02.08,89.36,-1.96,-2.15
2,WTI원유선물,2022.02.07,91.32,-0.99,-1.07
3,WTI원유선물,2022.02.04,92.31,2.04,2.26
4,WTI원유선물,2022.02.03,90.27,2.01,2.28
...,...,...,...,...,...
13111,두바이유선물,2006.01.17,60.25,2.45,4.24
13112,두바이유선물,2006.01.13,57.80,-0.31,-0.53
13113,두바이유선물,2006.01.12,58.11,0.42,0.73
13114,두바이유선물,2006.01.11,57.69,-0.38,-0.65


### 브렌트원유

In [68]:
codename = '브렌트원유선물'

In [69]:
import MySQLdb
db = MySQLdb.connect(host = 'localhost', user = 'root', password = '1234')
cursor = db.cursor()

In [70]:
sql = """
    use oil_price;
"""
cursor.execute(sql)

0

In [71]:
url = 'https://finance.daum.net/domestic/exchange/COMMODITY-%2FLCOc1'
driver.get(url)
time.sleep(0.5)

In [72]:
# 1~10
for _ in range(1):
    for page in range(2,12):
        html = driver.page_source
        df = pd.read_html(html)[0]
        df['전일비'] = df['전일비'].str.replace('▼','-').str.replace('▲','').astype('float')
        df['등락률'] = df['등락률'].str.replace('%','').astype(float)
        for i in range(len(df)):
            sql = f"""
                insert into daum values ('{codename}','{df.iloc[i,0]}',{df.iloc[i,1]}, {df.iloc[i,2]}, {df.iloc[i,3]})"""
            cursor.execute(sql)
        path = f'div.paging > a:nth-child({page})'
        driver.find_element_by_css_selector(path).click()
        time.sleep(0.5)
db.commit()

In [73]:
# 11~270
for _ in range(27):
    for page in range(4,14):
        html = driver.page_source
        df = pd.read_html(html)[0]
        df['전일비'] = df['전일비'].str.replace('▼','-').str.replace('▲','').astype('float')
        df['등락률'] = df['등락률'].str.replace('%','').astype(float)
        for i in range(len(df)):
            sql = f"""
                insert into daum values ('{codename}','{df.iloc[i,0]}',{df.iloc[i,1]}, {df.iloc[i,2]}, {df.iloc[i,3]})"""
            cursor.execute(sql)
        path = f'div.paging > a:nth-child({page})'
        driver.find_element_by_css_selector(path).click()
        time.sleep(0.5)
db.commit()

In [77]:
# 131~134
for _ in range(1):
    for page in range(10,17):
        html = driver.page_source
        df = pd.read_html(html)[0]
        df['전일비'] = df['전일비'].str.replace('▼','-').str.replace('▲','').astype('float')
        df['등락률'] = df['등락률'].str.replace('%','').str.replace(',','').astype(float)
        for i in range(len(df)):
            sql = f"""
                insert into daum values ('{codename}','{df.iloc[i,0]}',{df.iloc[i,1]}, {df.iloc[i,2]}, {df.iloc[i,3]})"""
            cursor.execute(sql)
        path = f'div.paging > a:nth-child({page})'
        driver.find_element_by_css_selector(path).click()
        time.sleep(0.5)
db.commit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.paging > a:nth-child(13)"}
  (Session info: chrome=98.0.4758.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00AEFDC3+2555331]
	Ordinal0 [0x00A877F1+2127857]
	Ordinal0 [0x00982E08+1060360]
	Ordinal0 [0x009AE49E+1238174]
	Ordinal0 [0x009AE69B+1238683]
	Ordinal0 [0x009D9252+1413714]
	Ordinal0 [0x009C7B54+1342292]
	Ordinal0 [0x009D75FA+1406458]
	Ordinal0 [0x009C7976+1341814]
	Ordinal0 [0x009A36B6+1193654]
	Ordinal0 [0x009A4546+1197382]
	GetHandleVerifier [0x00C89622+1619522]
	GetHandleVerifier [0x00D3882C+2336844]
	GetHandleVerifier [0x00B823E1+541697]
	GetHandleVerifier [0x00B81443+537699]
	Ordinal0 [0x00A8D18E+2150798]
	Ordinal0 [0x00A91518+2168088]
	Ordinal0 [0x00A91660+2168416]
	Ordinal0 [0x00A9B330+2208560]
	BaseThreadInitThunk [0x7714FA29+25]
	RtlGetAppContainerNamedObjectPath [0x772F7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x772F7A6E+238]


In [78]:
db.commit()

# 복기

In [79]:
tmpdf =pd.read_sql('select * from daum', db)

In [81]:
tmpdf = tmpdf.drop_duplicates()
tmpdf

,company,date,종가,전일비,등락률
0,WTI원유선물,2022.02.09,89.66,0.30,0.34
1,WTI원유선물,2022.02.08,89.36,-1.96,-2.15
2,WTI원유선물,2022.02.07,91.32,-0.99,-1.07
3,WTI원유선물,2022.02.04,92.31,2.04,2.26
4,WTI원유선물,2022.02.03,90.27,2.01,2.28
...,...,...,...,...,...
21614,브렌트원유선물,1990.01.08,20.52,-1.48,-6.73
21615,브렌트원유선물,1990.01.05,22.00,0.49,2.28
21616,브렌트원유선물,1990.01.04,21.51,-0.18,-0.83
21617,브렌트원유선물,1990.01.03,21.69,1.18,5.75


In [82]:
tmpdf[tmpdf['company']=='브렌트원유선물']

,company,date,종가,전일비,등락률
13116,브렌트원유선물,2022.02.09,91.55,0.77,0.85
13117,브렌트원유선물,2022.02.08,90.78,-1.91,-2.06
13118,브렌트원유선물,2022.02.07,92.69,-0.58,-0.62
13119,브렌트원유선물,2022.02.04,93.27,2.16,2.37
13120,브렌트원유선물,2022.02.03,91.11,1.64,1.83
...,...,...,...,...,...
21614,브렌트원유선물,1990.01.08,20.52,-1.48,-6.73
21615,브렌트원유선물,1990.01.05,22.00,0.49,2.28
21616,브렌트원유선물,1990.01.04,21.51,-0.18,-0.83
21617,브렌트원유선물,1990.01.03,21.69,1.18,5.75


In [83]:
tmpdf[tmpdf['company']=='두바이유선물']

,company,date,종가,전일비,등락률
8072,두바이유선물,2022.02.09,89.883,0.601,0.67
8073,두바이유선물,2022.02.08,89.282,-1.475,-1.63
8074,두바이유선물,2022.02.07,90.757,-0.299,-0.33
8075,두바이유선물,2022.02.04,91.056,1.900,2.13
8076,두바이유선물,2022.02.03,89.156,1.391,1.58
...,...,...,...,...,...
12635,두바이유선물,2006.01.30,61.430,0.540,0.89
12636,두바이유선물,2006.01.27,60.890,0.940,1.57
12637,두바이유선물,2006.01.26,59.950,0.630,1.06
12638,두바이유선물,2006.01.25,59.320,-1.470,-2.42


In [84]:
tmpdf[tmpdf['company']=='WTI원유선물']

,company,date,종가,전일비,등락률
0,WTI원유선물,2022.02.09,89.66,0.30,0.34
1,WTI원유선물,2022.02.08,89.36,-1.96,-2.15
2,WTI원유선물,2022.02.07,91.32,-0.99,-1.07
3,WTI원유선물,2022.02.04,92.31,2.04,2.26
4,WTI원유선물,2022.02.03,90.27,2.01,2.28
...,...,...,...,...,...
8067,WTI원유선물,1990.01.08,21.62,-1.46,-6.33
8068,WTI원유선물,1990.01.05,23.08,-0.33,-1.41
8069,WTI원유선물,1990.01.04,23.41,-0.27,-1.14
8070,WTI원유선물,1990.01.03,23.68,0.79,3.45


In [85]:
tmpdf.to_csv('daum_oil.csv',index=False)

In [91]:
url = 'https://dart.fss.or.kr/report/viewer.do?rcpNo=20211112000348&dcmNo=8271438&eleId=12&offset=54809&length=43207&dtd=dart3.xsd'
driver.get(url)

In [92]:
html = driver.page_source

In [94]:
pd.read_html(html)[0]

,유 형,품 목,구 분,2021년3분기말,2020년 (제21기),2019년(제20기)
0,상품,화장품안료,국내,-,-,-
1,상품,화장품안료,수입,"1,107,128 $473,530 EUR424,254","1,854,415 $1,039,587 EUR472,884","3,018,085 $1,949,113 EUR578,789"
2,상품,기타,국내,42872,65789,146553
3,상품,기타,수입,"65,779 $57,964","75,304$66,015","74,957$64,800"
4,소 계,소 계,국내,42872,65789,146553
5,소 계,소 계,수입,"1,172,907 $531,495 EUR424,254","1,929,719 $1,105,603 EUR472,884","3,093,042 $2,013,913 EUR578,789"
6,원재료,합성Mica,국내,1046778,1658334,1951342
7,원재료,합성Mica,수입,-,-,-
8,원재료,TiCl4,국내,1476899,1653289,1942233
9,원재료,TiCl4,수입,-,-,-
